In [1]:
import os
import glob
import workflow_utils as wf
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
if not os.getenv("GEMINI_API_KEY") and not os.getenv("OPENAI_API_KEY"):
    print("⚠️ 警告: 未检测到 API KEY，请检查环境变量或 workflow_utils 配置")

# --- 📁 全局路径配置 (修改这里即可全局生效) ---
CONFIG = {
    "pdf_dir": "./academic_papers",        # 原始 PDF
    "extract_dir": "./extracted_output",   # 中间产物 1: 提取的文本
    "llm_dir": "./llm_output",             # 中间产物 2: LLM 翻译结果
    "vis_dir": "./vis_output"              # 最终产物: HTML 和图片资源
}

# 自动创建目录
for d in CONFIG.values():
    os.makedirs(d, exist_ok=True)

print("✅ 环境初始化完成，目录已就绪。")

✅ 环境初始化完成，目录已就绪。


In [2]:
# --- 🎯 选择要处理的论文 (定义当前工作上下文) ---

# 1. 扫描 PDF
pdf_files = sorted(glob.glob(os.path.join(CONFIG["pdf_dir"], "*.pdf")))
if not pdf_files:
    print(f"❌ 未找到 PDF 文件，请放入 {CONFIG['pdf_dir']}")
    CURRENT_PROJECT = None
else:
    print("请选择要处理的文献：")
    for i, f in enumerate(pdf_files):
        print(f"[{i+1}] {os.path.basename(f)}")
    print("-" * 30)

    # 2. 用户输入 (如果想固定处理某文件，可以直接写 idx = 0)
    try:
        sel = input("请输入序号 (默认 1): ")
        idx = int(sel) - 1 if sel.strip() else 0
        
        if 0 <= idx < len(pdf_files):
            target_pdf = pdf_files[idx]
            raw_name = wf.sanitize_filename(target_pdf) # 获取不带后缀的文件名
            
            # --- 🔑 核心：构建所有路径字典 (State Dictionary) ---
            CURRENT_PROJECT = {
                "name": raw_name,
                "pdf_path": target_pdf,
                "context_path": os.path.join(CONFIG["extract_dir"], f"{raw_name}_context.txt"),
                
                # 产物 2: LLM 翻译结果
                "llm_result_path": os.path.join(CONFIG["llm_dir"], f"{raw_name}_llm_result.txt"),
                
                # 【新增】产物 2.5: 翻译过程缓存 (JSON)
                "llm_cache_path": os.path.join(CONFIG["llm_dir"], f"{raw_name}_llm_cache.json"),
                
                "vis_output_dir": os.path.join(CONFIG["vis_dir"], raw_name) 
            }
            
            print(f"\n✅ 当前锁定项目: 【{raw_name}】")
            print(f"   📄 文本路径将为: {CURRENT_PROJECT['context_path']}")
            print(f"   🤖 翻译路径将为: {CURRENT_PROJECT['llm_result_path']}")
        else:
            print("❌ 序号无效")
            CURRENT_PROJECT = None
    except Exception as e:
        print(f"❌ 选择出错: {e}")

请选择要处理的文献：
[1] A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective.pdf
[2] AlphaDrive_Unleashing_the_Power_of_VLMs_in_Autonomous_Driving_via_Reinforcement_Learning_and_Reasoning.pdf
[3] Deep_Reinforcement_Learning_IN_Autonomous_Car_Path_Planning_AND_Control_A_Survey.pdf
[4] Deep_Reinforcement_Learning_for_Autonomous_Driving_A_Survey.pdf
[5] Deep_Reinforcement_Learning_for_Autonomous_Driving_Challenges_and_Solutions.pdf
[6] Zhang_CarPlanner_Consistent_Auto-regressive_Trajectory_Planning_for_Large-Scale_Reinforcement_Learning_in_CVPR_2025_paper.pdf
------------------------------


请输入序号 (默认 1):  1



✅ 当前锁定项目: 【A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective】
   📄 文本路径将为: ./extracted_output/A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective_context.txt
   🤖 翻译路径将为: ./llm_output/A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective_llm_result.txt


In [3]:
# --- Step 1: 提取文本与资源 ---
if CURRENT_PROJECT:
    # 检查是否已有离线产物
    if os.path.exists(CURRENT_PROJECT["context_path"]):
        print(f"⚠️ 检测到已存在提取文件: {CURRENT_PROJECT['context_path']}")
        do_extract = input("是否重新提取? (y/N): ").lower() == 'y'
    else:
        do_extract = True

    if do_extract:
        print("🚀 开始提取布局与文本...")
        # 调用 utils
        full_text, txt_path, vis_dir, count = wf.extract_text_and_save_assets_smart(
            CURRENT_PROJECT["pdf_path"], 
            CONFIG["extract_dir"], 
            CONFIG["vis_dir"]
        )
        print(f"✅ Step 1 完成! 提取了 {count} 个资源。")
    else:
        print("⏭️ 跳过 Step 1，使用现有离线文件。")
else:
    print("❌ 请先运行 Cell 2 选择项目。")

⚠️ 检测到已存在提取文件: ./extracted_output/A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective_context.txt


是否重新提取? (y/N):  y


🚀 开始提取布局与文本...
📂 检测到历史标注记录: ./extracted_output/A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective/layout_config.json，正在加载...
💾 标注进度已保存至: ./extracted_output/A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective/layout_config.json
🧩 正在处理资源 (保存至: ./extracted_output/A_Survey_of_Reinforcement_Learning-Based_Motion_Planning_for_Autonomous_Driving_Lessons_Learned_from_a_Driving_Task_Perspective/assets)...
📝 提取正文文本...
🔍 [DEBUG] 开始处理 113 个文本块...
✅ Step 1 完成! 提取了 17 个资源。


In [ ]:
# --- Step 2: LLM 翻译 ---
if CURRENT_PROJECT:
    # 1. 检查输入是否存在
    if not os.path.exists(CURRENT_PROJECT["context_path"]):
        print(f"❌ 错误: 未找到上下文文件 ({CURRENT_PROJECT['context_path']})。")
        print("   请先运行 Step 1 进行提取。")
    else:
        # 2. 检查输出是否存在 (防止误触重跑消耗 Token)
        if os.path.exists(CURRENT_PROJECT["llm_result_path"]):
            print(f"⚠️ 检测到已有翻译结果。")
            do_trans = input("是否覆盖并重新翻译? (y/N): ").lower() == 'y'
        else:
            do_trans = True
            
        if do_trans:
            print("🚀 开始 LLM 翻译 (支持断点重试)...")
            # 注意：传入的是【文件路径】，utils 里会自动读取
            wf.run_smart_analysis(
                CURRENT_PROJECT["context_path"], 
                CURRENT_PROJECT["llm_result_path"],
                cache_path=CURRENT_PROJECT["llm_cache_path"]
            )
            print(f"✅ Step 2 完成! 结果已保存。")
        else:
            print("⏭️ 跳过 Step 2，使用现有翻译文件。")
else:
    print("❌ 请先运行 Cell 2 选择项目。")

🚀 开始 LLM 翻译 (支持断点重试)...
📋 [阶段一] 编排任务: 总片段 132 个

🚀 [阶段二] 开始推理 (剩余 132 个)...
   ⚡ Part 1/132 [META] ...

In [ ]:
# --- Step 3: 生成 HTML 报告 ---
from IPython.display import IFrame, display

if CURRENT_PROJECT:
    if not os.path.exists(CURRENT_PROJECT["llm_result_path"]):
        print("❌ 错误: 未找到翻译结果文件。请先运行 Step 2。")
    else:
        print("🚀 正在组装 HTML...")
        report_path = wf.generate_html_report(
            CURRENT_PROJECT["llm_result_path"], 
            CURRENT_PROJECT["vis_output_dir"]
        )
        
        print(f"✅ 报告生成完毕: {report_path}")
        display(IFrame(src=report_path, width="100%", height="800px"))
else:
    print("❌ 请先运行 Cell 2 选择项目。")